#### In this problem we use "Transfer Learning" of an Object Detector model to detect any object according to the problem in hand.

Here, We are particularly interested in detecting faces in a given image.

#### To use the model first, we need to import the model and its supporting files for the model to function. 

We see the below steps to import the model.

# ***I HAVE USED THE LATEST PYTHON 3 VERSION FOR THIS PROJECT***

### Import MobileNet model given in file `mn_model.py`

In [0]:
from google.colab import drive
from sklearn import preprocessing

In [2]:
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive/ComputerVision_with_CNNs/Face_Detection')

In [4]:
### Import MobileNet model given in file `mn_model.py`

from mn_model import mn_model

Using TensorFlow backend.


In [0]:
'''
Includes:
* A batch generator for SSD model training and inference which can perform online data agumentation
* An offline image processor that saves processed images and adjusted labels to disk

Copyright (C) 2017 Pierluigi Ferrari

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
'''

import sys
import skimage
from skimage import transform

import numpy as np
import cv2
import random
from sklearn.utils import shuffle 
from copy import deepcopy
from PIL import Image
import csv
import os
import scipy.misc as sm
from bs4 import BeautifulSoup
from keras.applications.imagenet_utils import preprocess_input
from keras import backend as K
import os
from keras.preprocessing import image
import matplotlib as mpl 
mpl.use('Agg')
from matplotlib import pyplot as plt

from termcolor import colored
#from matplotlib import pyplot as plt 
from tqdm import tqdm
#from __future__ import print_function


# Error : TypeError: can't pickle generator objects


#from scipy.misc import imresize

#resize_fct = imresize
#resize_fct = np.array(Image.fromarray(arr).resize())

bb_expanded = False


def save_bb(path, filename, results, prediction=True):
  
  # print filename
  _SIZ = 300 
  # img = image.load_img(filename, target_size=(224, 224))
  img = image.load_img(filename)
  
  img_height = img.height 
  img_width = img.width

  img = image.img_to_array(img)

  filename = filename.split("/")[-1]

  if(not prediction):
    filename = filename[:-4] + "_gt" + ".jpg"

  #fig,currentAxis = plt.subplots(1)
  currentAxis = plt.gca()

 # Get detections with confidence higher than 0.6.
  colors = plt.cm.hsv(np.linspace(0, 1, 21)).tolist()
  color_code = min(len(results), 16)
  print (colored("total number of bbs: %d" % len(results), "yellow"))
  for result in results:
    # Parse the outputs.

    if(prediction):
      det_label = result[0]
      det_conf = result[1]
      det_xmin = result[2]
      det_xmax = result[3]
      det_ymin = result[4]
      det_ymax = result[5]
    else :
      det_label = result[0]
      det_xmin = result[1] * img_width / _SIZ
      det_xmax = result[2] * img_width / _SIZ
      det_ymin = result[3] * img_height / _SIZ
      det_ymax = result[4] * img_height / _SIZ
    

    
    xmin = int(det_xmin)
    ymin = int(det_ymin)
    xmax = int(det_xmax)
    ymax = int(det_ymax)

    if(prediction):
      score = det_conf
    
    plt.imshow(img / 255.)
    
    label = int(int(det_label))

    #print label
    # label_name = "seq_to_class_name[label]"


    label_name = "face"
    # label_name = class_names[label]
    # print label_name 
    # print label

    if(prediction):
      display_txt = '{:0.2f}, {}'.format(score, label_name)
    else:
      display_txt = '{}'.format(label_name)

      
    # print (xmin, ymin, ymin, ymax)
    coords = (xmin, ymin), (xmax-xmin), (ymax-ymin)
    color_code = color_code-1 
    color = colors[0]
    currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor=color, linewidth=1))
    # currentAxis.text(xmin, ymin, display_txt, bbox={'facecolor':color, 'alpha':0.5})

  plt.savefig(path + filename)

  print ('saved' , path + filename)

  plt.clf()
  

def classify(img_path):
    img_ = io.imread(img_path)
    #if grayscale > RGB
    if(len(img_.shape)==2):
        img_ = np.repeat(np.expand_dims(img_,-1),3,axis=-1)
    if(img_.shape[-1]==4):
        img_ = img_[:,:,0:3]
    img_ = resize_fct(img_,tuple(input_shape[0:2]))[:,:,::-1] - 128. #BGR - 128.
    img_ = np.expand_dims(img_,0) 

    prediction_ = sess.run('Sigmoid:0', feed_dict={'input_1:0': img_})
    return prediction_, img_



# import threading
# class threadsafe_iter:
    # def __init__(self, it):
        # self.it = it
        # self.lock = threading.Lock()

    # def __iter__(self):
        # return self

    # def next(self):
        # with self.lock:
            # return self.it.next()

# def threadsafe_generator(f):
    # def g(*a, **kw):
        # return threadsafe_iter(f(*a, **kw))
    # return g

# def preprocess_mobilenet_input(x):
    # x /= 255.
    # x -= 0.5
    # x *= 2.
    # return x

# Image processing functions used by the generator to perform the following image manipulations:
# - Translation
# - Horizontal flip
# - Scaling
# - Brightness change
# - Histogram contrast equalization

def _translate(image, horizontal=(0,40), vertical=(0,10)):
    '''
    Randomly translate the input image horizontally and vertically.

    Arguments:
        image (array-like): The image to be translated.
        horizontal (int tuple, optinal): A 2-tuple `(min, max)` with the minimum
            and maximum horizontal translation. A random translation value will
            be picked from a uniform distribution over [min, max].
        vertical (int tuple, optional): Analog to `horizontal`.

    Returns:
        The translated image and the horzontal and vertical shift values.
    '''
    rows,cols,ch = image.shape

    x = np.random.randint(horizontal[0], horizontal[1]+1)
    y = np.random.randint(vertical[0], vertical[1]+1)
    x_shift = random.choice([-x, x])
    y_shift = random.choice([-y, y])

    M = np.float32([[1,0,x_shift],[0,1,y_shift]])
    return cv2.warpAffine(image, M, (cols, rows)), x_shift, y_shift

def _flip(image, orientation='horizontal'):
    '''
    Flip the input image horizontally or vertically.
    '''
    if orientation == 'horizontal':
        return cv2.flip(image, 1)
    else:
        return cv2.flip(image, 0)

def _scale(image, min=0.9, max=1.1):
    '''
    Scale the input image by a random factor picked from a uniform distribution
    over [min, max].

    Returns:
        The scaled image, the associated warp matrix, and the scaling value.
    '''

    rows,cols,ch = image.shape

    #Randomly select a scaling factor from the range passed.
    scale = np.random.uniform(min, max)

    M = cv2.getRotationMatrix2D((cols/2,rows/2), 0, scale)
    return cv2.warpAffine(image, M, (cols, rows)), M, scale

def _brightness(image, min=0.5, max=2.0):
    '''
    Randomly change the brightness of the input image.

    Protected against overflow.
    '''
    hsv = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)

    random_br = np.random.uniform(min,max)

    #To protect against overflow: Calculate a mask for all pixels
    #where adjustment of the brightness would exceed the maximum
    #brightness value and set the value to the maximum at those pixels.
    mask = hsv[:,:,2] * random_br > 255
    v_channel = np.where(mask, 255, hsv[:,:,2] * random_br)
    hsv[:,:,2] = v_channel

    return cv2.cvtColor(hsv,cv2.COLOR_HSV2RGB)

def histogram_eq(image):
    '''
    Perform histogram equalization on the input image.

    See https://en.wikipedia.org/wiki/Histogram_equalization.
    '''

    image1 = np.copy(image)

    image1[:,:,0] = cv2.equalizeHist(image1[:,:,0])
    image1[:,:,1] = cv2.equalizeHist(image1[:,:,1])
    image1[:,:,2] = cv2.equalizeHist(image1[:,:,2])

    return image1

class BatchGenerator:
    '''
    A generator to generate batches of samples and corresponding labels indefinitely.

    The labels are read from a CSV file.

    Shuffles the dataset consistently after each complete pass.

    Can perform image transformations for data conversion and data augmentation,
    for details please refer to the documentation of the `generate()` method.
    '''

    def __init__(self,
                 images_path,
                 include_classes='all',
                 box_output_format=['class_id', 'xmin', 'xmax', 'ymin', 'ymax']):
        '''
        Arguments:
            images_path (str): The filepath to the image samples.
            include_classes (list, optional): Either 'all' or a list of integers containing the class IDs that
                are to be included in the dataset. Defaults to 'all', in which case all boxes will be included
                in the dataset.
            box_output_format (list, optional): A list of five strings representing the desired order of the five
                items class ID, xmin, xmax, ymin, ymax in the generated data. The expected strings are
                'xmin', 'xmax', 'ymin', 'ymax', 'class_id'. If you want to train the model, this
                must be the order that the box encoding class requires as input. Defaults to
                `['class_id', 'xmin', 'xmax', 'ymin', 'ymax']`. Note that even though the parser methods are
                able to produce different output formats, the SSDBoxEncoder currently requires the format
                `['class_id', 'xmin', 'xmax', 'ymin', 'ymax']`. This list only specifies the five box parameters
                that are relevant as training targets, a list of filenames is generated separately.
        '''
        # These are the variables we always need
        self.images_path = images_path
        self.include_classes = include_classes
        self.box_output_format = box_output_format

        # These are the variables that we only need if we want to use parse_csv()
        self.labels_path = None
        self.input_format = None

        # These are the variables that we only need if we want to use parse_xml()
        self.annotations_path = None
        self.image_set_path = None
        self.image_set = None
        self.classes = None

        # The two variables below store the output from the parsers. This is the input for the generate() method
        # `self.filenames` is a list containing all file names of the image samples. Note that it does not contain the actual image files themselves.
        self.filenames = [] # All unique image filenames will go here
        # `self.labels` is a list containing one 2D Numpy array per image. For an image with `k` ground truth bounding boxes,
        # the respective 2D array has `k` rows, each row containing `(xmin, xmax, ymin, ymax, class_id)` for the respective bounding box.
        self.labels = [] # Each entry here will contain a 2D Numpy array with all the ground truth boxes for a given image

    def parse_csv(self,
                  labels_path=None,
                  input_format=None,
                  ret=False):
        '''
        Arguments:
            labels_path (str, optional): The filepath to a CSV file that contains one ground truth bounding box per line
                and each line contains the following six items: image file name, class ID, xmin, xmax, ymin, ymax.
                The six items do not have to be in a specific order, but they must be the first six columns of
                each line. The order of these items in the CSV file must be specified in `input_format`.
                The class ID is an integer greater than zero. Class ID 0 is reserved for the background class.
                `xmin` and `xmax` are the left-most and right-most absolute horizontal coordinates of the box,
                `ymin` and `ymax` are the top-most and bottom-most absolute vertical coordinates of the box.
                The image name is expected to be just the name of the image file without the directory path
                at which the image is located. Defaults to `None`.
            input_format (list, optional): A list of six strings representing the order of the six items
                image file name, class ID, xmin, xmax, ymin, ymax in the input CSV file. The expected strings
                are 'image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'. Defaults to `None`.
            ret (bool, optional): Whether or not the image filenames and labels are to be returned.
                Defaults to `False`.

        Returns:
            None by default, optionally the image filenames and labels.
        '''
        

        # If we get arguments in this call, set them
        if not labels_path is None: self.labels_path = labels_path
        if not input_format is None: self.input_format = input_format

        # Before we begin, make sure that we have a labels_path and an input_format
        if self.labels_path is None or self.input_format is None:
            raise ValueError("`labels_path` and/or `input_format` have not been set yet. You need to pass them as arguments.")

        # Erase data that might have been parsed before
        self.filenames = []
        self.labels = []

        # First, just read in the CSV file lines and sort them.

        data = []

        #print labels_path

        with open(self.labels_path) as csvfile:
            csvread = csv.reader(csvfile, delimiter=',')
            k = 0
            for i in csvread: # For every line (i.e for every bounding box) in the CSV file...
                if k == 0: # Skip the header row
                    k += 1
                    continue
                else:
                    if self.include_classes == 'all' or int(i[self.input_format.index('class_id')].strip()) in self.include_classes: # If the class_id is among the classes that are to be included in the dataset...
                        obj = [] # Store the box class and coordinates here
                        obj.append(i[self.input_format.index('image_name')].strip()) # Select the image name column in the input format and append its content to `obj`
                        for item in self.box_output_format: # For each item in the output format...
                            obj.append(int(i[self.input_format.index(item)].strip())) # ...select the respective column in the input format and append it to `obj`
                        data.append(obj)

        data = sorted(data) # The data needs to be sorted, otherwise the next step won't give the correct result

        # Now that we've made sure that the data is sorted by file names,
        # we can compile the actual samples and labels lists

        current_file = '' # The current image for which we're collecting the ground truth boxes
        current_labels = [] # The list where we collect all ground truth boxes for a given image
        for idx, i in enumerate(data):
            if current_file == '': # If this is the first image file
                current_file = i[0]
                current_labels.append(i[1:])
                if len(data) == 1: # If there is only one box in the CVS file
                    self.labels.append(np.stack(current_labels, axis=0))
                    self.filenames.append(current_file)
            else:
                if i[0] == current_file: # If this box (i.e. this line of the CSV file) belongs to the current image file
                    current_labels.append(i[1:])
                    if idx == len(data)-1: # If this is the last line of the CSV file
                        self.labels.append(np.stack(current_labels, axis=0))
                        self.filenames.append(current_file)
                else: # If this box belongs to a new image file
                    self.labels.append(np.stack(current_labels, axis=0))
                    self.filenames.append(current_file)
                    current_labels = []
                    current_file = i[0]
                    current_labels.append(i[1:])

        if ret: # In case we want to return these
            return self.filenames, self.labels

    def parse_xml(self,
                      annotations_path=None,
                      image_set_path=None,
                      image_set=None,
                      classes=['background',
                               'aeroplane', 'bicycle', 'bird', 'boat',
                               'bottle', 'bus', 'car', 'cat',
                               'chair', 'cow', 'diningtable', 'dog',
                               'horse', 'motorbike', 'person', 'pottedplant',
                               'sheep', 'sofa', 'train', 'tvmonitor'],
                      exclude_truncated=False,
                      exclude_difficult=False,
                      ret=False,
                      debug = False):
            '''
            This is a parser for the Pascal VOC datasets. It might be used for other datasets with minor changes to
            the code, but in its current form it expects the data format and XML tags of the Pascal VOC datasets.

            Arguments:
                annotations_path (str, optional): The path to the directory that contains the annotation XML files for
                    the images. The directory must contain one XML file per image and name of the XML file must be the
                    image ID. The content of the XML files must be in the Pascal VOC format. Defaults to `None`.
                image_set_path (str, optional): The path to the directory that contains a text file with the image
                    set to be loaded. Defaults to `None`.
                image_set (str, optional): The name of the image set text file to be loaded, ending in '.txt'.
                    This text file simply contains one image ID per line and nothing else. Defaults to `None`.
                classes (list, optional): A list containing the names of the object classes as found in the
                    `name` XML tags. Must include the class `background` as the first list item. The order of this list
                    defines the class IDs. Defaults to the list of Pascal VOC classes in alphabetical order.
                exclude_truncated (bool, optional): If `True`, excludes boxes that are labeled as 'truncated'.
                    Defaults to `False`.
                exclude_difficult (bool, optional): If `True`, excludes boxes that are labeled as 'difficult'.
                    Defaults to `False`.
                ret (bool, optional): Whether or not the image filenames and labels are to be returned.
                    Defaults to `False`.

            Returns:
                None by default, optionally the image filenames and labels.
            '''

            if not annotations_path is None: self.annotations_path = annotations_path
            if not image_set_path is None: self.image_set_path = image_set_path
            if not image_set is None: self.image_set = image_set
            if not classes is None: self.classes = classes

            # Erase data that might have been parsed before
            self.filenames = []
            self.labels = []

            data = np.load(annotations_path,allow_pickle= True).item()

            n_train_samples = len(data)
            train_cnt =0 
            for key in data:
                #img_path = "./images/"
                train_cnt = train_cnt + 1
                sys.stdout.flush()
                #print "reading training image {0} of {1}".format(train_cnt, n_train_samples)
                img_path = "./"
                img_name = data[key][1]
                image_id = key
                
                folder = None
                filename = img_path + img_name 
                
                #print  (filename)

                boxes = [] # We'll store all boxes for this image here
                
                n_objects = len(data[key]) - 3
                #print filename
                img_test = cv2.imread(filename)

                if(img_test is None):
                    continue

                

                height, width, channels = img_test.shape
                #print height, width, channels

                num_valid_objects = 0
                for obj in range(n_objects):

                    #class_id = data[key][3+obj][1]-1 # hicham's data has the -1 issue as all classes are incremented by 1
                    #playment data follows proper numbering for class so this code has to be changd for hicham's data
                    class_id = data[key][3+obj][1]
                    class_name = classes[class_id]
                    pose = None
                    truncated = False
                    difficult = False

                    #(x1,x2,y1,y2)
                    xmin = data[key][3+obj][0][0]
                    xmax = data[key][3+obj][0][1]
                    ymin = data[key][3+obj][0][2]
                    ymax = data[key][3+obj][0][3]

                    bb_width =(xmax - xmin) * float(512)/width
                    bb_height =(ymax - ymin) * float(512)/height 

                    if(bb_width > 8 and bb_height > 8): 
                        num_valid_objects +=1
                        # the image size after resizing will be too small for training 

                        # print filename
                        # print xmin, xmax, ymin, ymax

                        # extract bounding box and increase the size 
                        # xmin = max(data[key][3+obj][0][0] , 0) * (width-1)
                        # xmax = min(data[key][3+obj][0][1] , 1.0) * (width-1) 
                        # ymin = max(data[key][3+obj][0][2] , 0) * (height-1) 
                        # ymax = min(data[key][3+obj][0][3] , 1.0) * (height-1)  

                        
                        item_dict = {'folder': folder,
                                     'image_name': filename,
                                     'image_id': image_id,
                                     'class_name': class_name,
                                     'class_id': class_id,
                                     'pose': pose,
                                     'truncated': truncated,
                                     'difficult': difficult,
                                     'xmin': xmin,
                                     'ymin': ymin,
                                     'xmax': xmax,
                                     'ymax': ymax}
                          
                        #if(class_name == "text"):             
                        #    print class_name, class_id, filename
                            
                        #    cv2.rectangle(img_test, (xmin,ymin), (xmax, ymax), (255,0,0))           
                
                       
                        box = []
                        for item in self.box_output_format:
                            box.append(item_dict[item])
                        boxes.append(box)
                
                if(num_valid_objects > 0):        
                    self.filenames.append(filename)                   
                    self.labels.append(boxes)

                #if(class_name == "text"):
                    #cv2.imwrite("./test2/" +str(ymin)+str(ymax)+str(xmin)+str(xmax)+ filename[-5:-1]+ ".png", img_test)

            if ret:
                return self.filenames, self.labels


    #@threadsafe_generator        
    def generate(self,
                 batch_size=4,
                 train=True,
                 ssd_box_encoder=None,
                 equalize=False,
                 brightness=False,
                 flip=False,
                 translate=False,
                 scale=False,
                 random_crop=False,
                 crop=False,
                 resize=False,
                 gray=False,
                 limit_boxes=True,
                 include_thresh=0.3,
                 diagnostics=False):
        '''
        Generate batches of samples and corresponding labels indefinitely from
        lists of filenames and labels.

        Returns two numpy arrays, one containing the next `batch_size` samples
        from `filenames`, the other containing the corresponding labels from
        `labels`.

        Shuffles `filenames` and `labels` consistently after each complete pass.

        Can perform image transformations for data conversion and data augmentation.
        `resize`, `gray`, and `equalize` are image conversion tools and should be
        used consistently during training and inference. The remaining transformations
        serve for data augmentation. Each data augmentation process can set its own
        independent application probability. The transformations are performed
        in the order of their arguments, i.e. equalization is performed first,
        grayscale conversion is performed last.

        `prob` works the same way in all arguments in which it appears. It must be a float in [0,1]
        and determines the probability that the respective transform is applied to any given image.

        All conversions and transforms default to `False`.

        Arguments:
            batch_size (int, optional): The size of the batches to be generated. Defaults to 32.
            train (bool, optional): Whether or not the generator is used in training mode. If `True`, then the labels
                will be transformed into the format that the SSD cost function requires. Otherwise,
                the output format of the labels is identical to the input format. Defaults to `True`.
            ssd_box_encoder (SSDBoxEncoder, optional): Only required if `train = True`. An SSDBoxEncoder object
                to encode the ground truth labels to the required format for training an SSD model.
            equalize (bool, optional): If `True`, performs histogram equalization on the images.
                This can improve contrast and lead the improved model performance.
            brightness (tuple, optional): `False` or a tuple containing three floats, `(min, max, prob)`.
                Scales the brightness of the image by a factor randomly picked from a uniform
                distribution in the boundaries of `[min, max]`. Both min and max must be >=0.
            flip (float, optional): `False` or a float in [0,1], see `prob` above. Flip the image horizontally.
                The respective box coordinates are adjusted accordingly.
            translate (tuple, optional): `False` or a tuple, with the first two elements tuples containing
                two integers each, and the third element a float: `((min, max), (min, max), prob)`.
                The first tuple provides the range in pixels for horizontal shift of the image,
                the second tuple for vertical shift. The number of pixels to shift the image
                by is uniformly distributed within the boundaries of `[min, max]`, i.e. `min` is the number
                of pixels by which the image is translated at least. Both `min` and `max` must be >=0.
                The respective box coordinates are adjusted accordingly.
            scale (tuple, optional): `False` or a tuple containing three floats, `(min, max, prob)`.
                Scales the image by a factor randomly picked from a uniform distribution in the boundaries
                of `[min, max]`. Both min and max must be >=0.
            random_crop (tuple, optional): `False` or a tuple of four integers, `(height, width, min_1_object, max_#_trials)`,
                where `height` and `width` are the height and width of the patch that is to be cropped out at a random
                position in the input image. Note that `height` and `width` can be arbitrary - they are allowed to be larger
                than the image height and width, in which case the original image will be randomly placed on a black background
                canvas of size `(height, width)`. `min_1_object` is either 0 or 1. If 1, there must be at least one detectable
                object remaining in the image for the crop to be valid, and if 0, crops with no detectable objects left in the
                image patch are allowed. `max_#_trials` is only relevant if `min_1_object == 1` and sets the maximum number
                of attempts to get a valid crop. If no valid crop was obtained within this maximum number of attempts,
                the respective image will be removed from the batch without replacement (i.e. for each removed image, the batch
                will be one sample smaller). Defaults to `False`.
            crop (tuple, optional): `False` or a tuple of four integers, `(crop_top, crop_bottom, crop_left, crop_right)`,
                with the number of pixels to crop off of each side of the images.
                The targets are adjusted accordingly. Note: Cropping happens before resizing.
            resize (tuple, optional): `False` or a tuple of 2 integers for the desired output
                size of the images in pixels. The expected format is `(width, height)`.
                The box coordinates are adjusted accordingly. Note: Resizing happens after cropping.
            gray (bool, optional): If `True`, converts the images to grayscale.
            limit_boxes (bool, optional): If `True`, limits box coordinates to stay within image boundaries
                post any transformation. This should always be set to `True`, even if you set `include_thresh`
                to 0. I don't even know why I made this an option. If this is set to `False`, you could
                end up with some boxes that lie entirely outside the image boundaries after a given transformation
                and such boxes would of course not make any sense and have a strongly adverse effect on the learning.
            include_thresh (float, optional): Only relevant if `limit_boxes` is `True`. Determines the minimum
                fraction of the area of a ground truth box that must be left after limiting in order for the box
                to still be included in the batch data. If set to 0, all boxes are kept except those which lie
                entirely outside of the image bounderies after limiting. If set to 1, only boxes that did not
                need to be limited at all are kept. Defaults to 0.3.
            diagnostics (bool, optional): If `True`, yields three additional output items:
                1) A list of the image file names in the batch.
                2) An array with the original, unaltered images.
                3) A list with the original, unaltered labels.
                This can be useful for diagnostic purposes. Defaults to `False`. Only works if `train = True`.

        Yields:
            The next batch as a tuple containing a Numpy array that contains the images and a python list
            that contains the corresponding labels for each image as 2D Numpy arrays. The output format
            of the labels is according to the `box_output_format` that was specified in the constructor.
        '''

        self.filenames, self.labels = shuffle(self.filenames, self.labels) # Shuffle the data before we begin
        current = 0


        # Find out the indices of the box coordinates in the label data
        xmin = self.box_output_format.index('xmin')
        xmax = self.box_output_format.index('xmax')
        ymin = self.box_output_format.index('ymin')
        ymax = self.box_output_format.index('ymax')

        resize_function = "scipy"
        resize_select =1

        while True:

            batch_X, batch_y = [], []

            #Shuffle the data after each complete pass
            if current >= len(self.filenames):
                self.filenames, self.labels = shuffle(self.filenames, self.labels)
                current = 0

            for filename in self.filenames[current:current+batch_size]:

                #print os.path.join(self.images_path, filename)
                
                img = cv2.imread(filename)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                batch_X.append(img)

                #img_width = resize[0]
                #img_height = resize[1]
                #print img_width, img_height     


                #img = Image.load_img(os.path.join(self.images_path, filename), target_size=(img_width, img_height))
                #img_opencv = cv2.cvtColor(numpy.array(img), cv2.COLOR_RGB2BGR)
                #batch_X.append(np.array(img_opencv))

                #with Image.open('{}'.format(os.path.join(self.images_path, filename)), target_size = (img_height, img_width)) as img:
                    # convert the image to 
                #    img_opencv = cv2.cvtColor(numpy.array(img), cv2.COLOR_RGB2BGR)
                #    batch_X.append(np.array(img))

                    #print np.array(img).shape
                    #print np.array(batch_X).shape   


            batch_y = deepcopy(self.labels[current:current+batch_size])

            #print self.filenames[current]
            #print batch_y[0]

            this_filenames = self.filenames[current:current+batch_size] # The filenames of the files in the current batch

            if diagnostics:
                original_images = np.copy(batch_X) # The original, unaltered images
                original_labels = deepcopy(batch_y) # The original, unaltered labels

            current += batch_size

            # At this point we're done producing the batch. Now perform some
            # optional image transformations:

            batch_items_to_remove = [] # In case we need to remove any images from the batch because of failed random cropping, store their indices in this list

            for i in range(len(batch_X)):

                img_height, img_width, ch = batch_X[i].shape

                # print "====>"
                # print img_height, img_width
                batch_y[i] = np.array(batch_y[i]) # Convert labels into an array (in case it isn't one already), otherwise the indexing below breaks

                #print "filename :", this_filenames[i]
                #print "annotations :", batch_y[i]
                

                #print "batch_y[i][:,(xmin,xmax)]", batch_y[i][:,(xmin,xmax)]    
                if equalize:
                    batch_X[i] = histogram_eq(batch_X[i])

                if brightness:
                    p = np.random.uniform(0,1)
                    if p >= (1-brightness[2]):
                        batch_X[i] = _brightness(batch_X[i], min=brightness[0], max=brightness[1])

                # Could easily be extended to also allow vertical flipping, but I'm not convinced of the
                # usefulness of vertical flipping either empirically or theoretically, so I'm going for simplicity.
                # If you want to allow vertical flipping, just change this function to pass the respective argument
                # to `_flip()`.
                if flip:
                    p = np.random.uniform(0,1)
                    if p >= (1-flip):
                        batch_X[i] = _flip(batch_X[i])
                        #print "xmax, xmin", xmin, xmax
                        #print "y[i].shape", batch_y[i].shape
                        batch_y[i][:,[xmin,xmax]] = img_width - batch_y[i][:,[xmax,xmin]] # xmin and xmax are swapped when mirrored

                if translate:
                    p = np.random.uniform(0,1)
                    if p >= (1-translate[2]):
                        # Translate the image and return the shift values so that we can adjust the labels
                        batch_X[i], xshift, yshift = _translate(batch_X[i], translate[0], translate[1])
                        # Adjust the labels
                        batch_y[i][:,[xmin,xmax]] += xshift
                        batch_y[i][:,[ymin,ymax]] += yshift
                        # Limit the box coordinates to lie within the image boundaries
                        if limit_boxes:
                            before_limiting = deepcopy(batch_y[i])
                            x_coords = batch_y[i][:,[xmin,xmax]]
                            x_coords[x_coords >= img_width] = img_width - 1
                            x_coords[x_coords < 0] = 0
                            batch_y[i][:,[xmin,xmax]] = x_coords
                            y_coords = batch_y[i][:,[ymin,ymax]]
                            y_coords[y_coords >= img_height] = img_height - 1
                            y_coords[y_coords < 0] = 0
                            batch_y[i][:,[ymin,ymax]] = y_coords
                            # Some objects might have gotten pushed so far outside the image boundaries in the transformation
                            # process that they don't serve as useful training examples anymore, because too little of them is
                            # visible. We'll remove all boxes that we had to limit so much that their area is less than
                            # `include_thresh` of the box area before limiting.
                            before_area = (before_limiting[:,xmax] - before_limiting[:,xmin]) * (before_limiting[:,ymax] - before_limiting[:,ymin])
                            after_area = (batch_y[i][:,xmax] - batch_y[i][:,xmin]) * (batch_y[i][:,ymax] - batch_y[i][:,ymin])
                            if include_thresh == 0: batch_y[i] = batch_y[i][after_area > include_thresh * before_area] # If `include_thresh == 0`, we want to make sure that boxes with area 0 get thrown out, hence the ">" sign instead of the ">=" sign
                            else: batch_y[i] = batch_y[i][after_area >= include_thresh * before_area] # Especially for the case `include_thresh == 1` we want the ">=" sign, otherwise no boxes would be left at all

                if scale:
                    p = np.random.uniform(0,1)
                    if p >= (1-scale[2]):
                        # Rescale the image and return the transformation matrix M so we can use it to adjust the box coordinates
                        batch_X[i], M, scale_factor = _scale(batch_X[i], scale[0], scale[1])
                        # Adjust the box coordinates
                        # Transform two opposite corner points of the rectangular boxes using the transformation matrix `M`
                        toplefts = np.array([batch_y[i][:,xmin], batch_y[i][:,ymin], np.ones(batch_y[i].shape[0])])
                        bottomrights = np.array([batch_y[i][:,xmax], batch_y[i][:,ymax], np.ones(batch_y[i].shape[0])])
                        new_toplefts = (np.dot(M, toplefts)).T
                        new_bottomrights = (np.dot(M, bottomrights)).T
                        batch_y[i][:,[xmin,ymin]] = new_toplefts.astype(np.int)
                        batch_y[i][:,[xmax,ymax]] = new_bottomrights.astype(np.int)
                        # Limit the box coordinates to lie within the image boundaries
                        if limit_boxes and (scale_factor > 1): # We don't need to do any limiting in case we shrunk the image
                            before_limiting = deepcopy(batch_y[i])
                            x_coords = batch_y[i][:,[xmin,xmax]]
                            x_coords[x_coords >= img_width] = img_width - 1
                            x_coords[x_coords < 0] = 0
                            batch_y[i][:,[xmin,xmax]] = x_coords
                            y_coords = batch_y[i][:,[ymin,ymax]]
                            y_coords[y_coords >= img_height] = img_height - 1
                            y_coords[y_coords < 0] = 0
                            batch_y[i][:,[ymin,ymax]] = y_coords
                            # Some objects might have gotten pushed so far outside the image boundaries in the transformation
                            # process that they don't serve as useful training examples anymore, because too little of them is
                            # visible. We'll remove all boxes that we had to limit so much that their area is less than
                            # `include_thresh` of the box area before limiting.
                            before_area = (before_limiting[:,xmax] - before_limiting[:,xmin]) * (before_limiting[:,ymax] - before_limiting[:,ymin])
                            after_area = (batch_y[i][:,xmax] - batch_y[i][:,xmin]) * (batch_y[i][:,ymax] - batch_y[i][:,ymin])
                            if include_thresh == 0: batch_y[i] = batch_y[i][after_area > include_thresh * before_area] # If `include_thresh == 0`, we want to make sure that boxes with area 0 get thrown out, hence the ">" sign instead of the ">=" sign
                            else: batch_y[i] = batch_y[i][after_area >= include_thresh * before_area] # Especially for the case `include_thresh == 1` we want the ">=" sign, otherwise no boxes would be left at all

                if random_crop:

                    #random crop to be done only for 30% time 
                    p = np.random.uniform(0,1)

                    if(p < 0.3):
                        # Compute how much room we have in both dimensions to make a random crop.
                        # A negative number here means that we want to crop out a patch that is larger than the original image in the respective dimension,
                        # in which case we will create a black background canvas onto which we will randomly place the image.
                        y_range = img_height - random_crop[0]
                        x_range = img_width - random_crop[1]
                        # Keep track of the number of trials and of whether or not the most recent crop contains at least one object
                        min_1_object_fulfilled = False
                        trial_counter = 0
                        while (not min_1_object_fulfilled) and (trial_counter < random_crop[3]):
                            # Select a random crop position from the possible crop positions
                            if y_range >= 0: crop_ymin = np.random.randint(0, y_range + 1) # There are y_range + 1 possible positions for the crop in the vertical dimension
                            else: crop_ymin = np.random.randint(0, -y_range + 1) # The possible positions for the image on the background canvas in the vertical dimension
                            if x_range >= 0: crop_xmin = np.random.randint(0, x_range + 1) # There are x_range + 1 possible positions for the crop in the horizontal dimension
                            else: crop_xmin = np.random.randint(0, -x_range + 1) # The possible positions for the image on the background canvas in the horizontal dimension
                            # Perform the crop
                            if y_range >= 0 and x_range >= 0: # If the patch to be cropped out is smaller than the original image in both dimenstions, we just perform a regular crop
                                # Crop the image
                                patch_X = np.copy(batch_X[i][crop_ymin:crop_ymin+random_crop[0], crop_xmin:crop_xmin+random_crop[1]])
                                # Translate the box coordinates into the new coordinate system: Cropping shifts the origin by `(crop_ymin, crop_xmin)`
                                patch_y = np.copy(batch_y[i])
                                patch_y[:,[ymin,ymax]] -= crop_ymin
                                patch_y[:,[xmin,xmax]] -= crop_xmin
                                # Limit the box coordinates to lie within the new image boundaries
                                if limit_boxes:
                                    # Both the x- and y-coordinates might need to be limited
                                    before_limiting = np.copy(patch_y)
                                    y_coords = patch_y[:,[ymin,ymax]]
                                    y_coords[y_coords < 0] = 0
                                    y_coords[y_coords >= random_crop[0]] = random_crop[0] - 1
                                    patch_y[:,[ymin,ymax]] = y_coords
                                    x_coords = patch_y[:,[xmin,xmax]]
                                    x_coords[x_coords < 0] = 0
                                    x_coords[x_coords >= random_crop[1]] = random_crop[1] - 1
                                    patch_y[:,[xmin,xmax]] = x_coords
                            elif y_range >= 0 and x_range < 0: # If the crop is larger than the original image in the horizontal dimension only,...
                                # Crop the image
                                patch_X = np.copy(batch_X[i][crop_ymin:crop_ymin+random_crop[0]]) # ...crop the vertical dimension just as before,...
                                canvas = np.zeros((random_crop[0], random_crop[1], patch_X.shape[2]), dtype=np.uint8) # ...generate a blank background image to place the patch onto,...
                                canvas[:, crop_xmin:crop_xmin+img_width] = patch_X # ...and place the patch onto the canvas at the random `crop_xmin` position computed above.
                                patch_X = canvas
                                # Translate the box coordinates into the new coordinate system: In this case, the origin is shifted by `(crop_ymin, -crop_xmin)`
                                patch_y = np.copy(batch_y[i])
                                patch_y[:,[ymin,ymax]] -= crop_ymin
                                patch_y[:,[xmin,xmax]] += crop_xmin
                                # Limit the box coordinates to lie within the new image boundaries
                                if limit_boxes:
                                    # Only the y-coordinates might need to be limited
                                    before_limiting = np.copy(patch_y)
                                    y_coords = patch_y[:,[ymin,ymax]]
                                    y_coords[y_coords < 0] = 0
                                    y_coords[y_coords >= random_crop[0]] = random_crop[0] - 1
                                    patch_y[:,[ymin,ymax]] = y_coords
                            elif y_range < 0 and x_range >= 0: # If the crop is larger than the original image in the vertical dimension only,...
                                # Crop the image
                                patch_X = np.copy(batch_X[i][:,crop_xmin:crop_xmin+random_crop[1]]) # ...crop the horizontal dimension just as in the first case,...
                                canvas = np.zeros((random_crop[0], random_crop[1], patch_X.shape[2]), dtype=np.uint8) # ...generate a blank background image to place the patch onto,...
                                canvas[crop_ymin:crop_ymin+img_height, :] = patch_X # ...and place the patch onto the canvas at the random `crop_ymin` position computed above.
                                patch_X = canvas
                                # Translate the box coordinates into the new coordinate system: In this case, the origin is shifted by `(-crop_ymin, crop_xmin)`
                                patch_y = np.copy(batch_y[i])
                                patch_y[:,[ymin,ymax]] += crop_ymin
                                patch_y[:,[xmin,xmax]] -= crop_xmin
                                # Limit the box coordinates to lie within the new image boundaries
                                if limit_boxes:
                                    # Only the x-coordinates might need to be limited
                                    before_limiting = np.copy(patch_y)
                                    x_coords = patch_y[:,[xmin,xmax]]
                                    x_coords[x_coords < 0] = 0
                                    x_coords[x_coords >= random_crop[1]] = random_crop[1] - 1
                                    patch_y[:,[xmin,xmax]] = x_coords
                            else:  # If the crop is larger than the original image in both dimensions,...
                                patch_X = np.copy(batch_X[i])
                                canvas = np.zeros((random_crop[0], random_crop[1], patch_X.shape[2]), dtype=np.uint8) # ...generate a blank background image to place the patch onto,...
                                canvas[crop_ymin:crop_ymin+img_height, crop_xmin:crop_xmin+img_width] = patch_X # ...and place the patch onto the canvas at the random `(crop_ymin, crop_xmin)` position computed above.
                                patch_X = canvas
                                # Translate the box coordinates into the new coordinate system: In this case, the origin is shifted by `(-crop_ymin, -crop_xmin)`
                                patch_y = np.copy(batch_y[i])
                                patch_y[:,[ymin,ymax]] += crop_ymin
                                patch_y[:,[xmin,xmax]] += crop_xmin
                                # Note that no limiting is necessary in this case
                            # Some objects might have gotten pushed so far outside the image boundaries in the transformation
                            # process that they don't serve as useful training examples anymore, because too little of them is
                            # visible. We'll remove all boxes that we had to limit so much that their area is less than
                            # `include_thresh` of the box area before limiting.
                            if limit_boxes and (y_range >= 0 or x_range >= 0):
                                before_area = (before_limiting[:,xmax] - before_limiting[:,xmin]) * (before_limiting[:,ymax] - before_limiting[:,ymin])
                                after_area = (patch_y[:,xmax] - patch_y[:,xmin]) * (patch_y[:,ymax] - patch_y[:,ymin])
                                if include_thresh == 0: patch_y = patch_y[after_area > include_thresh * before_area] # If `include_thresh == 0`, we want to make sure that boxes with area 0 get thrown out, hence the ">" sign instead of the ">=" sign
                                else: patch_y = patch_y[after_area >= include_thresh * before_area] # Especially for the case `include_thresh == 1` we want the ">=" sign, otherwise no boxes would be left at all
                            trial_counter += 1 # We've just used one of our trials
                            # Check if we have found a valid crop
                            if random_crop[2] == 0: # If `min_1_object == 0`, break out of the while loop after the first loop because we are fine with whatever crop we got
                                batch_X[i] = patch_X # The cropped patch becomes our new batch item
                                batch_y[i] = patch_y # The adjusted boxes become our new labels for this batch item
                                # Update the image size so that subsequent transformations can work correctly
                                img_height = random_crop[0]
                                img_width = random_crop[1]
                                break
                            elif len(patch_y) > 0: # If we have at least one object left, this crop is valid and we can stop
                                min_1_object_fulfilled = True
                                batch_X[i] = patch_X # The cropped patch becomes our new batch item
                                batch_y[i] = patch_y # The adjusted boxes become our new labels for this batch item
                                # Update the image size so that subsequent transformations can work correctly
                                img_height = random_crop[0]
                                img_width = random_crop[1]
                            elif (trial_counter >= random_crop[3]) and (not i in batch_items_to_remove): # If we've reached the trial limit and still not found a valid crop, remove this image from the batch
                                batch_items_to_remove.append(i)

                if crop:
                    # Crop the image
                    batch_X[i] = np.copy(batch_X[i][crop[0]:img_height-crop[1], crop[2]:img_width-crop[3]])
                    # Translate the box coordinates into the new coordinate system if necessary: The origin is shifted by `(crop[0], crop[2])` (i.e. by the top and left crop values)
                    # If nothing was cropped off from the top or left of the image, the coordinate system stays the same as before
                    if crop[0] > 0:
                        batch_y[i][:,[ymin,ymax]] -= crop[0]
                    if crop[2] > 0:
                        batch_y[i][:,[xmin,xmax]] -= crop[2]
                    # Update the image size so that subsequent transformations can work correctly
                    img_height -= crop[0] + crop[1]
                    img_width -= crop[2] + crop[3]
                    # Limit the box coordinates to lie within the new image boundaries
                    if limit_boxes:
                        before_limiting = np.copy(batch_y[i])
                        # We only need to check those box coordinates that could possibly have been affected by the cropping
                        # For example, if we only crop off the top and/or bottom of the image, there is no need to check the x-coordinates
                        if crop[0] > 0:
                            y_coords = batch_y[i][:,[ymin,ymax]]
                            y_coords[y_coords < 0] = 0
                            batch_y[i][:,[ymin,ymax]] = y_coords
                        if crop[1] > 0:
                            y_coords = batch_y[i][:,[ymin,ymax]]
                            y_coords[y_coords >= img_height] = img_height - 1
                            batch_y[i][:,[ymin,ymax]] = y_coords
                        if crop[2] > 0:
                            x_coords = batch_y[i][:,[xmin,xmax]]
                            x_coords[x_coords < 0] = 0
                            batch_y[i][:,[xmin,xmax]] = x_coords
                        if crop[3] > 0:
                            x_coords = batch_y[i][:,[xmin,xmax]]
                            x_coords[x_coords >= img_width] = img_width - 1
                            batch_y[i][:,[xmin,xmax]] = x_coords
                        # Some objects might have gotten pushed so far outside the image boundaries in the transformation
                        # process that they don't serve as useful training examples anymore, because too little of them is
                        # visible. We'll remove all boxes that we had to limit so much that their area is less than
                        # `include_thresh` of the box area before limiting.
                        before_area = (before_limiting[:,xmax] - before_limiting[:,xmin]) * (before_limiting[:,ymax] - before_limiting[:,ymin])
                        after_area = (batch_y[i][:,xmax] - batch_y[i][:,xmin]) * (batch_y[i][:,ymax] - batch_y[i][:,ymin])
                        if include_thresh == 0: batch_y[i] = batch_y[i][after_area > include_thresh * before_area] # If `include_thresh == 0`, we want to make sure that boxes with area 0 get thrown out, hence the ">" sign instead of the ">=" sign
                        else: batch_y[i] = batch_y[i][after_area >= include_thresh * before_area] # Especially for the case `include_thresh == 1` we want the ">=" sign, otherwise no boxes would be left at all

                        
                
                if resize:
                    #print "coming here"
                    #print "before"
                    #print batch_y[i][:,[ymin,ymax]]

                    #print "===>resize:enabled"

                    #print (batch_y[i][:,[xmin,xmax]], batch_y[i][:,[ymin,ymax]])
                    
                    # resize using scipy function 

                    # use different types of scaling for images to improve the performance with different preprocessors 



                    if(resize_function == "scipy"):
                        #batch_X[i] = sm.imresize(batch_X[i], resize)
                        batch_X[i] = skimage.transform.resize(batch_X[i], resize)
                        #print "scipy resizing is used"
                    elif (resize_function == "pil"):
                        img_pil = Image.fromarray(batch_X[i])
                        img_pil = img_pil.resize(resize)
                        batch_X[i] = np.asarray(img_pil)
                    elif (resize_function == "cv"):
                        cv2.resize(batch_X[i], resize)
                    elif (resize_function == "hybrid"):
                        if(resize_select ==1):
                            #resize_function = "pil"
                            img_pil = Image.fromarray(batch_X[i])
                            img_pil = img_pil.resize(resize)
                            batch_X[i] = np.asarray(img_pil)
                            #print "---pil---"
                        elif(resize_select == 2):
                            #resize_function = "cv"
                            batch_X[i] = cv2.resize(batch_X[i], resize)
                            #print "---cv---"
                        else:
                            #resize_function = "scipy"
                            #print "---scipy---"
                            #batch_X[i] = sm.imresize(batch_X[i], resize)
                            batch_X[i] = skimage.transform.resize(batch_X[i], resize)
                    else:
                        #batch_X[i] = sm.imresize(batch_X[i], resize)                  
                        batch_X[i] = skimage.transform.resize(batch_X[i], resize)
                        #print "---scipy---"
                    resize_select = resize_select +1

                    if(resize_select %6 ==0):
                            resize_select = 1

                       
                    #batch_X[i] = cv2.resize_fct(batch_X[i], dsize=resize, interpolation= cv2.INTER_LINEAR);
                    #kernel = np.ones((3,3), np.float32)/9
                    #dst = cv2.filter2D(batch_X[i], -1, kernel)
                    #batch_X[i] = dst; 

                    # print (this_filenames[i])

                    # print ("before resizing")
                    # print (batch_y[i])



                    # print ("SIZE before: ")
                    # print (xmin, xmax, ymin, ymax)
                    # print (xmax-xmin, ymax-ymin)

                    batch_y[i][:,[xmin,xmax]] = (batch_y[i][:,[xmin,xmax]] * (float(resize[0]) / img_width)).astype(np.int)
                    batch_y[i][:,[ymin,ymax]] = (batch_y[i][:,[ymin,ymax]] * (float(resize[1]) / img_height)).astype(np.int)
                    

                    # results = batch_y[i] 

                    # for result in results:
                    #     i_xmin = result[1]
                    #     i_xmax = result[2]
                    #     i_ymin = result[3]
                    #     i_ymax = result[4]

                    #     print (i_xmax-i_xmin, i_ymax-i_ymin)

                    # save_bb("./annotations/", this_filenames[i], batch_y[i], prediction=False)

                    # print img_width, img_height
                    # print ("after resizing")
                    # print (batch_y[i])

                    #print batch_y[i][:,[ymin,ymax]]
                    #print "after"  
                    img_width, img_height = resize # Updating these at this point is unnecessary, but it's one fewer source of error if this method gets expanded in the future

                if gray:
                    batch_X[i] = np.expand_dims(cv2.cvtColor(batch_X[i], cv2.COLOR_RGB2GRAY), 3)

            # If any batch items need to be removed because of failed random cropping, remove them now.
            for j in sorted(batch_items_to_remove, reverse=True):
                batch_X.pop(j)
                batch_y.pop(j) # This isn't efficient, but this should hopefully not need to be done often anyway

            if train: # During training we need the encoded labels instead of the format that `batch_y` has
                if ssd_box_encoder is None:
                    raise ValueError("`ssd_box_encoder` cannot be `None` in training mode.")
                y_true = ssd_box_encoder.encode_y(batch_y) # Encode the labels into the `y_true` tensor that the cost function needs

            # CAUTION: Converting `batch_X` into an array will result in an empty batch if the images have varying sizes.
            #          At this point, all images have to have the same size, otherwise you will get an error during training.




            if train:
                if diagnostics:
                    yield (np.array(batch_X), y_true, batch_y, this_filenames, original_images, original_labels)
                else:
                    #print "============================", np.ndim((np.array(batch_X), y_true))
                    #print y_true
                    #print "Batch Shape :", np.array(batch_X).shape

                    #print "y_true", y_true
                    #print "batch_y", batch_y

                    yield (np.array(batch_X), y_true)

            else:
                yield ((np.array(batch_X)), batch_y, this_filenames)

 


    def get_filenames_labels(self):
        '''
        Returns:
            The list of filenames and the list of labels.
        '''
        return self.filenames, self.labels

    def get_n_samples(self):
        '''
        Returns:
            The number of image files in the initialized dataset.
        '''
        return len(self.filenames)

    def process_offline(self,
                        dest_path='',
                        start=0,
                        stop='all',
                        crop=False,
                        equalize=False,
                        brightness=False,
                        flip=False,
                        translate=False,
                        scale=False,
                        resize=False,
                        gray=False,
                        limit_boxes=True,
                        include_thresh=0.3,
                        diagnostics=False):
        '''
        Perform offline image processing.

        This function the same image processing capabilities as the generator function above,
        but it performs the processing on all items in `filenames` starting at index `start`
        until index `stop` and saves the processed images to disk. The labels are adjusted
        accordingly.

        Processing images offline is useful to reduce the amount of work done by the batch
        generator and thus can speed up training. For example, transformations that are performed
        on all images in a deterministic way, such as resizing or cropping, should be done offline.

        Arguments:
            dest_path (str, optional): The destination directory where the processed images
                and `labels.csv` should be saved, ending on a slash.
            start (int, optional): The inclusive start index from which onward to process the
                items in `filenames`. Defaults to 0.
            stop (int, optional): The exclusive stop index until which to process the
                items in `filenames`. Defaults to 'all', meaning to process all items until the
                end of the list.

        For a description of the other arguments, please refer to the documentation of `generate_batch()` above.

        Returns:
            `None`, but saves all processed images as JPEG files to the specified destination
            directory and generates a `labels.csv` CSV file that is saved to the same directory.
            The format of the lines in the destination CSV file is the same as that of the
            source CSV file, i.e. `[frame, xmin, xmax, ymin, ymax, class_id]`.
        '''

        import gc

        targets_for_csv = []
        if stop == 'all':
            stop = len(self.filenames)

        if diagnostics:
            processed_images = []
            original_images = []
            processed_labels = []

        # Find out the indices of the box coordinates in the label data
        xmin = self.box_output_format.index('xmin')
        xmax = self.box_output_format.index('xmax')
        ymin = self.box_output_format.index('ymin')
        ymax = self.box_output_format.index('ymax')

        for k, filename in enumerate(self.filenames[start:stop]):
            i = k + start
            with Image.open('{}'.format(os.path.join(self.images_path, filename))) as img:
                image = np.array(img)
            targets = np.copy(self.labels[i])

            if diagnostics:
                original_images.append(image)

            img_height, img_width, ch = image.shape

            if equalize:
                image = histogram_eq(image)

            if brightness:
                p = np.random.uniform(0,1)
                if p >= (1-brightness[2]):
                    image = _brightness(image, min=brightness[0], max=brightness[1])

            # Could easily be extended to also allow vertical flipping, but I'm not convinced of the
            # usefulness of vertical flipping either empirically or theoretically, so I'm going for simplicity.
            # If you want to allow vertical flipping, just change this function to pass the respective argument
            # to `_flip()`.
            if flip:
                p = np.random.uniform(0,1)
                if p >= (1-flip):
                    image = _flip(image)
                    targets[:,[0,1]] = img_width - targets[:,[1,0]] # xmin and xmax are swapped when mirrored

            if translate:
                p = np.random.uniform(0,1)
                if p >= (1-translate[2]):
                    image, xshift, yshift = _translate(image, translate[0], translate[1])
                    targets[:,[0,1]] += xshift
                    targets[:,[2,3]] += yshift
                    if limit_boxes:
                        before_limiting = np.copy(targets)
                        x_coords = targets[:,[0,1]]
                        x_coords[x_coords >= img_width] = img_width - 1
                        x_coords[x_coords < 0] = 0
                        targets[:,[0,1]] = x_coords
                        y_coords = targets[:,[2,3]]
                        y_coords[y_coords >= img_height] = img_height - 1
                        y_coords[y_coords < 0] = 0
                        targets[:,[2,3]] = y_coords
                        # Some objects might have gotten pushed so far outside the image boundaries in the transformation
                        # process that they don't serve as useful training examples anymore, because too little of them is
                        # visible. We'll remove all boxes that we had to limit so much that their area is less than
                        # `include_thresh` of the box area before limiting.
                        before_area = (before_limiting[:,1] - before_limiting[:,0]) * (before_limiting[:,3] - before_limiting[:,2])
                        after_area = (targets[:,1] - targets[:,0]) * (targets[:,3] - targets[:,2])
                        targets = targets[after_area >= include_thresh * before_area]

            if scale:
                p = np.random.uniform(0,1)
                if p >= (1-scale[2]):
                    image, M, scale_factor = _scale(image, scale[0], scale[1])
                    # Transform two opposite corner points of the rectangular boxes using the transformation matrix `M`
                    toplefts = np.array([targets[:,0], targets[:,2], np.ones(targets.shape[0])])
                    bottomrights = np.array([targets[:,1], targets[:,3], np.ones(targets.shape[0])])
                    new_toplefts = (np.dot(M, toplefts)).T
                    new_bottomrights = (np.dot(M, bottomrights)).T
                    targets[:,[0,2]] = new_toplefts.astype(np.int)
                    targets[:,[1,3]] = new_bottomrights.astype(np.int)
                    if limit_boxes and (scale_factor > 1): # We don't need to do any limiting in case we shrunk the image
                        before_limiting = np.copy(targets)
                        x_coords = targets[:,[0,1]]
                        x_coords[x_coords >= img_width] = img_width - 1
                        x_coords[x_coords < 0] = 0
                        targets[:,[0,1]] = x_coords
                        y_coords = targets[:,[2,3]]
                        y_coords[y_coords >= img_height] = img_height - 1
                        y_coords[y_coords < 0] = 0
                        targets[:,[2,3]] = y_coords
                        # Some objects might have gotten pushed so far outside the image boundaries in the transformation
                        # process that they don't serve as useful training examples anymore, because too little of them is
                        # visible. We'll remove all boxes that we had to limit so much that their area is less than
                        # `include_thresh` of the box area before limiting.
                        before_area = (before_limiting[:,1] - before_limiting[:,0]) * (before_limiting[:,3] - before_limiting[:,2])
                        after_area = (targets[:,1] - targets[:,0]) * (targets[:,3] - targets[:,2])
                        targets = targets[after_area >= include_thresh * before_area]

            if crop:
                image = image[crop[0]:img_height-crop[1], crop[2]:img_width-crop[3]]
                if limit_boxes: # Adjust boxes affected by cropping and remove those that will no longer be in the image
                    before_limiting = np.copy(targets)
                    if crop[0] > 0:
                        y_coords = targets[:,[2,3]]
                        y_coords[y_coords < crop[0]] = crop[0]
                        targets[:,[2,3]] = y_coords
                    if crop[1] > 0:
                        y_coords = targets[:,[2,3]]
                        y_coords[y_coords >= (img_height - crop[1])] = img_height - crop[1] - 1
                        targets[:,[2,3]] = y_coords
                    if crop[2] > 0:
                        x_coords = targets[:,[0,1]]
                        x_coords[x_coords < crop[2]] = crop[2]
                        targets[:,[0,1]] = x_coords
                    if crop[3] > 0:
                        x_coords = targets[:,[0,1]]
                        x_coords[x_coords >= (img_width - crop[3])] = img_width - crop[3] - 1
                        targets[:,[0,1]] = x_coords
                    # Some objects might have gotten pushed so far outside the image boundaries in the transformation
                    # process that they don't serve as useful training examples anymore, because too little of them is
                    # visible. We'll remove all boxes that we had to limit so much that their area is less than
                    # `include_thresh` of the box area before limiting.
                    before_area = (before_limiting[:,1] - before_limiting[:,0]) * (before_limiting[:,3] - before_limiting[:,2])
                    after_area = (targets[:,1] - targets[:,0]) * (targets[:,3] - targets[:,2])
                    targets = targets[after_area >= include_thresh * before_area]
                # Now adjust the box coordinates for the new image size post cropping
                if crop[0] > 0:
                    targets[:,[2,3]] -= crop[0]
                if crop[2] > 0:
                    targets[:,[0,1]] -= crop[2]
                img_height -= crop[0] - crop[1]
                img_width -= crop[2] - crop[3]

            if resize:
                image = cv2.resize(image, dsize=resize)
                targets[:,[0,1]] = (targets[:,[0,1]] * (resize[0] / img_width)).astype(np.int)
                targets[:,[2,3]] = (targets[:,[2,3]] * (resize[1] / img_height)).astype(np.int)

            if gray:
                image = np.expand_dims(cv2.cvtColor(image, cv2.COLOR_RGB2GRAY), 3)

            if diagnostics:
                processed_images.append(image)
                processed_labels.append(targets)

            img = Image.fromarray(image.astype(np.uint8))
            img.save('{}{}'.format(dest_path, filename), 'JPEG', quality=90)
            del image
            del img
            gc.collect()

            # Transform the labels back to the original CSV file format:
            # One line per ground truth box, i.e. possibly multiple lines per image
            for target in targets:
                target = list(target)
                target = [filename] + target
                targets_for_csv.append(target)

        with open('{}labels.csv'.format(dest_path), 'w', newline='') as csvfile:
            labelswriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
            labelswriter.writerow(['frame', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'])
            labelswriter.writerows(targets_for_csv)

        if diagnostics:
            print("Image processing completed.")
            return np.array(processed_images), np.array(original_images), np.array(targets_for_csv), processed_labels
        else:
            print("Image processing completed.")








### Import the BatchGenerator and SSDLoss functions in given files `face_generator.py`, `keras_ssd_loss` and `ssd_box_encode_decode_utils.py` as well, used in MobileNet model

In [0]:
#### Import the BatchGenerator and SSDLoss functions as well, used in MobileNet model

#from face_generator import BatchGenerator
from keras_ssd_loss import SSDLoss
from ssd_box_encode_decode_utils import SSDBoxEncoder, decode_y, decode_y2

In [0]:
import warnings
warnings.filterwarnings("ignore")

from keras.preprocessing import image

from keras.optimizers import Adam, SGD, Nadam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard, LearningRateScheduler
from keras.callbacks import Callback
from keras import backend as K 
from keras.models import load_model
from math import ceil 
import numpy as np 
from termcolor import colored

import keras
from keras import applications
from keras.models import Sequential, Model
from keras.layers import Input, Convolution2D, Dropout, Flatten, Dense
from keras import optimizers
from keras import backend as k
from keras.callbacks import ModelCheckpoint, EarlyStopping

from mn_model import mn_model
#from face_generator import BatchGenerator
from keras_ssd_loss import SSDLoss
from ssd_box_encode_decode_utils import SSDBoxEncoder, decode_y, decode_y2

# training parameters
from keras import backend as K
import scipy.misc as sm


## Set the parameters for the model

#### We need to customize the model parameters according to our problem as given below.

#### Set n_classes (no.of classes) = 2, as we are interested in only face detection. 
#### `Face` will be one class and everything else comes under other class (we can call it as `background`)

#### Set class_names = ["background", "face"]

In [0]:
img_height = 512
img_width = 512

img_channels = 3

n_classes = 2 
class_names = ["background","face"]

scales = [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05] # anchorboxes for coco dataset
aspect_ratios = [[0.5, 1.0, 2.0],
                 [1.0/3.0, 0.5, 1.0, 2.0, 3.0],
                 [1.0/3.0, 0.5, 1.0, 2.0, 3.0],
                 [1.0/3.0, 0.5, 1.0, 2.0, 3.0],
                 [0.5, 1.0, 2.0],
                 [0.5, 1.0, 2.0]] # The anchor box aspect ratios used in the original SSD300
two_boxes_for_ar1 = True
limit_boxes = True # Whether or not you want to limit the anchor boxes to lie entirely within the image boundaries
variances = [0.1, 0.1, 0.2, 0.2] # The variances by which the encoded target coordinates are scaled as in the original implementation
coords = 'centroids' # Whether the box coordinates to be used as targets for the model should be in the 'centroids' or 'minmax' format, see documentation
normalize_coords = True

det_model_path = "./"

### Now, we have imported the model and its dependencies. The next thing is to import the dataset for the model to train on. For this, we are using the WIDER FACE dataset. 

#### To make the dataset available follow the steps given below.


1. Create a folder in your google drive for this project. 

2. Download the train and test dataset files given in .zip format into your drive folder you created for the project in step-1.

3. Set the project path variable according to the folders you created to use for this project in your google drive. 

      `project_path = "/content/drive/My Drive/DLCP/"`

4. Now, as we mount the drive the images will be available to use for training and testing but in zip format.

5. So, lets extract the images from the zipfiles by using the code given of zipfile module.







In [0]:
from google.colab import drive

In [10]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
project_path = "/content/drive/My Drive/ComputerVision_with_CNNs/Face_Detection/"

In [0]:
train_images_path = project_path + 'WIDER_train.zip'
test_images_path = project_path + 'WIDER_val.zip'

In [0]:
# NO NEED TO RUN THE BELOW STEPS AGAIN AS THE FILES ARE ALREADY UNZIPPED

In [0]:
import zipfile
archive = zipfile.ZipFile(train_images_path, 'r')
archive.extractall()

In [0]:
archive = zipfile.ZipFile(test_images_path, 'r')
archive.extractall()

### Now, the images are available. The next thing we need is to get the labels for these images, so that we can use this information while training for detecting faces with the given model using transfer learning. 

#### Follow the below steps to get those labels available.


Load the  '' `wider_train_small.npy`'' file given to check the information given about the dataset. In this file you can see the information about each image in the dataset in a list with following elements:
      

        1.   Image filename (str)
        2.   Image filename (str)
        3.   Image size (list) [height, width]
        4.   List of bounding box co-ordinates and Class label (list) [[a,b,c,d], Class label, ...]
        
        where,
        a,b,c,d are the four co-ordinates of the bounding box
        Class label is the position of object as mentioned in `class_names` list above.

In [0]:
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

In [0]:
data = np.load('./wider_train_small.npy').item()

In [0]:
# restore np.load for future normal usage
#np.load = np_load_old

In [17]:
data

{'0--Parade/0_Parade_Parade_0_127.jpg': ['WIDER_train/images/0--Parade/0_Parade_Parade_0_127.jpg',
  'WIDER_train/images/0--Parade/0_Parade_Parade_0_127.jpg',
  [300, 300],
  [[247, 262, 320, 339], 1],
  [[497, 514, 344, 367], 1],
  [[432, 445, 304, 322], 1],
  [[226, 242, 315, 334], 1],
  [[204, 219, 308, 325], 1],
  [[181, 196, 304, 320], 1],
  [[159, 173, 293, 310], 1],
  [[319, 332, 313, 332], 1],
  [[289, 302, 306, 325], 1],
  [[260, 272, 302, 319], 1],
  [[235, 248, 297, 314], 1],
  [[213, 225, 290, 306], 1],
  [[192, 204, 285, 300], 1],
  [[174, 185, 279, 294], 1],
  [[375, 390, 305, 325], 1],
  [[345, 360, 305, 323], 1],
  [[323, 336, 294, 311], 1],
  [[297, 309, 292, 308], 1],
  [[268, 280, 287, 303], 1],
  [[247, 259, 279, 295], 1],
  [[225, 237, 274, 291], 1],
  [[203, 212, 268, 283], 1],
  [[178, 189, 266, 278], 1],
  [[187, 198, 248, 261], 1],
  [[208, 218, 254, 266], 1],
  [[214, 224, 239, 250], 1],
  [[135, 147, 289, 304], 1],
  [[115, 126, 283, 297], 1],
  [[98, 109, 27

In [18]:
### Printed first element to check the above given information.

for key in data:
    print (key)
    #print (data[key])
    break

52--Photographers/52_Photographers_photographertakingphoto_52_582.jpg


In [19]:
for key in data:
    #print (key)
    print (data[key])
    break

['WIDER_train/images/52--Photographers/52_Photographers_photographertakingphoto_52_582.jpg', 'WIDER_train/images/52--Photographers/52_Photographers_photographertakingphoto_52_582.jpg', [300, 300], [[21, 667, 9, 655], 1]]


As we can see from the above output all the information mentioned above is there for all the images.

### Now, load the files `wider_trian.npy` and `wider_val.npy`

In [0]:
train_data = 'wider_train_small.npy'
test_data = 'wider_val_small.npy'

a = np.load(train_data)
b = np.load(test_data)

In [21]:
a

array({'52--Photographers/52_Photographers_photographertakingphoto_52_582.jpg': ['WIDER_train/images/52--Photographers/52_Photographers_photographertakingphoto_52_582.jpg', 'WIDER_train/images/52--Photographers/52_Photographers_photographertakingphoto_52_582.jpg', [300, 300], [[21, 667, 9, 655], 1]], '4--Dancing/4_Dancing_Dancing_4_108.jpg': ['WIDER_train/images/4--Dancing/4_Dancing_Dancing_4_108.jpg', 'WIDER_train/images/4--Dancing/4_Dancing_Dancing_4_108.jpg', [300, 300], [[476, 526, 332, 366], 1], [[739, 759, 478, 504], 1], [[974, 995, 481, 506], 1], [[792, 809, 478, 499], 1], [[844, 860, 490, 512], 1], [[900, 918, 499, 522], 1], [[686, 703, 482, 505], 1], [[639, 655, 488, 509], 1], [[590, 604, 483, 504], 1], [[553, 566, 492, 510], 1], [[500, 515, 480, 500], 1], [[238, 251, 480, 497], 1], [[283, 295, 490, 508], 1], [[322, 335, 480, 498], 1], [[345, 357, 484, 501], 1], [[361, 375, 484, 505], 1], [[199, 214, 473, 493], 1], [[161, 174, 477, 495], 1], [[77, 91, 475, 496], 1], [[34, 47, 

In [22]:
b

array({'4--Dancing/4_Dancing_Dancing_4_514.jpg': ['WIDER_val/images/4--Dancing/4_Dancing_Dancing_4_514.jpg', 'WIDER_val/images/4--Dancing/4_Dancing_Dancing_4_514.jpg', [300, 300], [[384, 448, 108, 220], 1], [[514, 600, 24, 148], 1]], '57--Angler/57_Angler_peoplefishing_57_153.jpg': ['WIDER_val/images/57--Angler/57_Angler_peoplefishing_57_153.jpg', 'WIDER_val/images/57--Angler/57_Angler_peoplefishing_57_153.jpg', [300, 300], [[206, 354, 246, 424], 1]], '22--Picnic/22_Picnic_Picnic_22_561.jpg': ['WIDER_val/images/22--Picnic/22_Picnic_Picnic_22_561.jpg', 'WIDER_val/images/22--Picnic/22_Picnic_Picnic_22_561.jpg', [300, 300], [[392, 494, 504, 629], 1], [[209, 324, 459, 632], 1]], '47--Matador_Bullfighter/47_Matador_Bullfighter_Matador_Bullfighter_47_617.jpg': ['WIDER_val/images/47--Matador_Bullfighter/47_Matador_Bullfighter_Matador_Bullfighter_47_617.jpg', 'WIDER_val/images/47--Matador_Bullfighter/47_Matador_Bullfighter_Matador_Bullfighter_47_617.jpg', [300, 300], [[171, 711, 11, 695], 1]],

### Now, call the imported model with the given parameters and freeze all the layers in the model with names not having ''`detection`'' word as prefix.

As we are not training the model from scratch, we are freezing all the above layers in the model having only last few layers while training to update their weights according to the problem in hand. This is called as **Transfer Learning**.

In [23]:
# build the keras model
# this model is not retrained, we are doing it from scratch 

K.clear_session()
model, model_layer, img_input, predictor_sizes = mn_model(image_size=(img_height, img_width, img_channels), 
                                                                      n_classes = n_classes,
                                                                      min_scale = None, 
                                                                      max_scale = None, 
                                                                      scales = scales, 
                                                                      aspect_ratios_global = None, 
                                                                      aspect_ratios_per_layer = aspect_ratios, 
                                                                      two_boxes_for_ar1= two_boxes_for_ar1, 
                                                                      limit_boxes=limit_boxes, 
                                                                      variances= variances, 
                                                                      coords=coords, 
                                                                      normalize_coords=normalize_coords)

W0825 16:19:50.148840 140168952248192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0825 16:19:50.151317 140168952248192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0825 16:19:50.202229 140168952248192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0825 16:19:50.205281 140168952248192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W082

====> Model Specific data
====> Height, Width, Channels : 512 512 3


W0825 16:19:53.112928 140168952248192 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.



In [24]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
lambda1 (Lambda)                (None, 512, 512, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda2 (Lambda)                (None, 512, 512, 3)  0           lambda1[0][0]                    
__________________________________________________________________________________________________
lambda3 (Lambda)                (None, 512, 512, 3)  0           lambda2[0][0]                    
__________________________________________________________________________________________________
conv1 (Con

#### Write code to freeze all the layers in the above model with names not having ''`detection`'' word as prefix.

In [25]:
#for layer in model.layers:
#  if('detection' not in layer.name):
#    layer.trainable = False

i = 0
for layer in model.layers[:]:
  if(not layer.name.startswith("detection")):
    layer.trainable = False
    i = i+1
    
for layer in model.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

print (i)

input_1
False
lambda1
False
lambda2
False
lambda3
False
conv1
False
conv1_bn
False
conv1_relu
False
conv_dw_1
False
conv_dw_1_bn
False
conv_dw_1_relu
False
conv_pw_1
False
conv_pw_1_bn
False
conv_pw_1_relu
False
conv_dw_2
False
conv_dw_2_bn
False
conv_dw_2_relu
False
conv_pw_2
False
conv_pw_2_bn
False
conv_pw_2_relu
False
conv_dw_3
False
conv_dw_3_bn
False
conv_dw_3_relu
False
conv_pw_3
False
conv_pw_3_bn
False
conv_pw_3_relu
False
conv_dw_4
False
conv_dw_4_bn
False
conv_dw_4_relu
False
conv_pw_4
False
conv_pw_4_bn
False
conv_pw_4_relu
False
conv_dw_5
False
conv_dw_5_bn
False
conv_dw_5_relu
False
conv_pw_5
False
conv_pw_5_bn
False
conv_pw_5_relu
False
conv_dw_6
False
conv_dw_6_bn
False
conv_dw_6_relu
False
conv_pw_6
False
conv_pw_6_bn
False
conv_pw_6_relu
False
conv_dw_7
False
conv_dw_7_bn
False
conv_dw_7_relu
False
conv_pw_7
False
conv_pw_7_bn
False
conv_pw_7_relu
False
conv_dw_8
False
conv_dw_8_bn
False
conv_dw_8_relu
False
conv_pw_8
False
conv_pw_8_bn
False
conv_pw_8_relu
False
conv

### After making the model ready for transfer learning, load the weights of the model given in file ''`mobilenet_1_0_224_tf.h5`''

In [0]:
model.load_weights(project_path + "mobilenet_1_0_224_tf.h5",by_name=True)

#### Using the functions given in the model, we are trying to divide the dataset into train and validation samples. Run the below code.

In [0]:
# restore np.load for future normal usage
np.load = np_load_old

In [28]:
batch_size = 4
ssd_box_encoder = SSDBoxEncoder(img_height=img_height,
                                img_width=img_width,
                                n_classes=n_classes, 
                                predictor_sizes=predictor_sizes,
                                min_scale=None,
                                max_scale=None,
                                scales=scales,
                                aspect_ratios_global=None,
                                aspect_ratios_per_layer=aspect_ratios,
                                two_boxes_for_ar1=two_boxes_for_ar1,
                                limit_boxes=limit_boxes,
                                variances=variances,
                                pos_iou_threshold=0.5,
                                neg_iou_threshold=0.2,
                                coords=coords,
                                normalize_coords=normalize_coords)

train_dataset = BatchGenerator(images_path=train_data, 
                include_classes='all', 
                box_output_format = ['class_id', 'xmin', 'xmax', 'ymin', 'ymax'])

print ("==>TRAINING DATA")
print ("==> Parsing XML files ...")

train_dataset.parse_xml(
                  annotations_path=train_data,
                  image_set_path='None',
                  image_set='None',
                  classes = class_names, 
                  #classes = class_names_ohe_reshape,
                  exclude_truncated=False,
                  exclude_difficult=False,
                  ret=False, 
                  debug = False)
print("==>Parsing XML Finished.")

print ("==>Generate training batches...")
train_generator = train_dataset.generate(
                 batch_size=batch_size,
                 train=True,
                 ssd_box_encoder=ssd_box_encoder,
                 equalize=True,
                 brightness=(0.5,2,0.5),
                 flip=0.5,
                 translate=((0, 20), (0, 30), 0.5),
                 scale=(0.75, 1.2, 0.5),
                 crop=False,
                 #random_crop = (img_height,img_width,1,3), 
                 random_crop=False,
                 resize=(img_height, img_width),
                 #resize=False,
                 gray=False,
                 limit_boxes=True,
                 include_thresh=0.4,
                 diagnostics=False)

print ("==>Training batch generation complete")

n_train_samples = train_dataset.get_n_samples()

print ("==>Total number of training samples = {}".format(n_train_samples))

# Now repeat above steps for validation data 

print ("==>VALIDATION")

val_dataset = BatchGenerator(images_path=test_data, include_classes='all', 
                box_output_format = ['class_id', 'xmin', 'xmax', 'ymin', 'ymax'])

print ("==> Parsing XML files ...")


val_dataset.parse_xml(
                  annotations_path=test_data,
                  image_set_path='None',
                  image_set='None',
                  classes = class_names, 
                  #classes = class_names_ohe_reshape,
                  exclude_truncated=False,
                  exclude_difficult=False,
                  ret=False, 
                  debug = False)


print("==>Parsing XML Finished.")


print ("==>Generate training batches...")
val_generator = val_dataset.generate(
                 batch_size=batch_size,
                 train=True,
                 ssd_box_encoder=ssd_box_encoder,
                 equalize=False,
                 brightness=False,
                 flip=False,
                 translate=False,
                 scale=False,
                 crop=False,
                 #random_crop = (img_height,img_width,1,3), 
                 random_crop=False, 
                 resize=(img_height, img_width), 
                 #resize=False, 
                 gray=False,
                 limit_boxes=True,
                 include_thresh=0.4,
                 diagnostics=False)


print ("==>Training batch generation complete")

n_val_samples = val_dataset.get_n_samples()

print ("==>Total number of validation samples = {}".format(n_val_samples))

==>TRAINING DATA
==> Parsing XML files ...
==>Parsing XML Finished.
==>Generate training batches...
==>Training batch generation complete
==>Total number of training samples = 128
==>VALIDATION
==> Parsing XML files ...
==>Parsing XML Finished.
==>Generate training batches...
==>Training batch generation complete
==>Total number of validation samples = 60


### Now, lets setup things for training by initilaizing required variables like learning rate, epochs, optimizer and loss function(SSDLoss) to compile the model

In [0]:
# setting up training 

# batch_size and no.of epochs

batch_size = 4
num_epochs = 100

#Learning rate
base_lr = 0.002

# Optimizer
adam = Adam(lr=base_lr, beta_1=0.9, beta_2=0.999, epsilon=1e-6, decay = 0.0)

# Loss
ssd_loss = SSDLoss(neg_pos_ratio=2, n_neg_min=0, alpha=1.0, beta = 1.0)

# Compile
model.compile(optimizer=adam, loss=ssd_loss.compute_loss, metrics=['accuracy'])

### Lets add early stopping and model checkpoint layers on validation loss with some patience values and using fit_generator function to train the model on data generated batch-by-batch by a Python generator, `train_generator` object as generator.


We are using checkpoint to save the best model based on validation accuracy.

#### Write code for early_stopping and model_checkpoint layers. Using model.fit_generator train the model and save the best weight file.

In [41]:
#import pdb; pdb.set_trace()

checkpoint = ModelCheckpoint("facedet_best.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

model.fit_generator(
train_generator,
samples_per_epoch = n_train_samples,
epochs = num_epochs,
validation_data = val_generator,
nb_val_samples = n_val_samples,
callbacks = [checkpoint, early])



Epoch 1/100
128/128 [==============================] - 126s 982ms/step - loss: 1.5871 - acc: 0.1183 - val_loss: 2.0303 - val_acc: 0.1312

Epoch 00001: val_acc improved from -inf to 0.13122, saving model to facedet_best.h5
Epoch 2/100
128/128 [==============================] - 112s 872ms/step - loss: 1.5480 - acc: 0.1210 - val_loss: 2.0855 - val_acc: 0.0950

Epoch 00002: val_acc did not improve from 0.13122
Epoch 3/100
128/128 [==============================] - 114s 888ms/step - loss: 1.5702 - acc: 0.1239 - val_loss: 1.9998 - val_acc: 0.2100

Epoch 00003: val_acc improved from 0.13122 to 0.20996, saving model to facedet_best.h5
Epoch 4/100
128/128 [==============================] - 112s 872ms/step - loss: 1.5628 - acc: 0.1239 - val_loss: 2.0116 - val_acc: 0.0405

Epoch 00004: val_acc did not improve from 0.20996
Epoch 5/100
128/128 [==============================] - 112s 879ms/step - loss: 1.5481 - acc: 0.1273 - val_loss: 1.9691 - val_acc: 0.1910

Epoch 00005: val_acc did not improve fr

### Load the best saved model from above step and check predictions for test data using test_generator object to generate batches.

#### Write code in the below cell to load best saved model in the above step.

In [0]:
model.load_weights('facedet_best.h5')

### Use the below function to plot the boundingbox in the test image to show the predictions.

In [0]:
def save_bb(path, filename, results, prediction=True):
  
  # print filename

  img = image.load_img(filename, target_size=(img_height, img_width))
  img = image.img_to_array(img)

  filename = filename.split("/")[-1]

  if(not prediction):
    filename = filename[:-4] + "_gt" + ".jpg"

  #fig,currentAxis = plt.subplots(1)
  currentAxis = plt.gca()

 # Get detections with confidence higher than 0.6.
  colors = plt.cm.hsv(np.linspace(0, 1, 25)).tolist()
  color_code = min(len(results), 16)
  print (colored("total number of bbs: %d" % len(results), "yellow"))
  for result in results:
    # Parse the outputs.

    if(prediction):
      det_label = result[0]
      det_conf = result[1]
      det_xmin = result[2]
      det_xmax = result[3]
      det_ymin = result[4]
      det_ymax = result[5]
    else :
      det_label = result[0]
      det_xmin = result[1]
      det_xmax = result[2]
      det_ymin = result[3]
      det_ymax = result[4]

    xmin = int(det_xmin)
    ymin = int(det_ymin)
    xmax = int(det_xmax)
    ymax = int(det_ymax)

    if(prediction):
      score = det_conf
    
    plt.imshow(img / 255.)
    
    label = int(int(det_label))
    label_name = class_names[label]
    # print label_name 
    # print label

    if(prediction):
      display_txt = '{:0.2f}'.format(score)
    else:
      display_txt = '{}'.format(label_name)

      
    # print (xmin, ymin, ymin, ymax)
    coords = (xmin, ymin), (xmax-xmin), (ymax-ymin)
    color_code = color_code-1 
    color = colors[color_code]
    currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor=color, linewidth=2))
    currentAxis.text(xmin, ymin, display_txt, bbox={'facecolor':color, 'alpha':0.2})

  # y
  currentAxis.axes.get_yaxis().set_visible(False)
  # x
  currentAxis.axes.get_xaxis().set_visible(False)
  plt.savefig(path + filename, bbox_inches='tight')

  print ('saved' , path + filename)

  plt.clf()


#### Run the below code to create a folder with name output_test and get the predictions for the test images using model.predict()

In [44]:
!mkdir output_test

mkdir: cannot create directory ‘output_test’: File exists


In [45]:
from keras.preprocessing import image
from matplotlib import pyplot as plt

test_size = 4
test_generator = val_dataset.generate(
                 batch_size=test_size,
                 train=False,
                 ssd_box_encoder=ssd_box_encoder,
                 equalize=False,
                 brightness=False,
                 flip=False,
                 translate=False,
                 scale=False,
                 crop=False,
                 #random_crop = (img_height,img_width,1,3), 
                 random_crop=False, 
                 resize=(img_height, img_width), 
                 #resize=False,
                 gray=False,
                 limit_boxes=True,
                 include_thresh=0.4,
                 diagnostics=False)

print (colored("done.", "green"))

print (colored("now predicting...", "yellow"))

_CONF = 0.60 
_IOU = 0.15

for i in range(test_size):
    X, y, filenames = next(test_generator)

    y_pred = model.predict(X)
    y_pred_decoded = decode_y2(y_pred,
                             confidence_thresh=_CONF,
                            iou_threshold=_IOU,
                            top_k='all',
                            input_coords=coords,
                            normalize_coords=normalize_coords,
                            img_height=img_height,
                            img_width=img_width)


    np.set_printoptions(suppress=True)

    save_bb("./output_test/", filenames[i], y_pred_decoded[i])
    save_bb("./output_test/", filenames[i], y[i], prediction=False)


done.
now predicting...
total number of bbs: 0
saved ./output_test/2_Demonstration_Demonstration_Or_Protest_2_306.jpg
total number of bbs: 10
saved ./output_test/2_Demonstration_Demonstration_Or_Protest_2_306_gt.jpg
total number of bbs: 0
saved ./output_test/2_Demonstration_Political_Rally_2_219.jpg
total number of bbs: 1
saved ./output_test/2_Demonstration_Political_Rally_2_219_gt.jpg
total number of bbs: 0
saved ./output_test/58_Hockey_icehockey_puck_58_285.jpg
total number of bbs: 7
saved ./output_test/58_Hockey_icehockey_puck_58_285_gt.jpg
total number of bbs: 0
saved ./output_test/41_Swimming_Swimming_41_580.jpg
total number of bbs: 1
saved ./output_test/41_Swimming_Swimming_41_580_gt.jpg


In the above step all the test images along with predictions are stored in output_test folder in this notebook environment. You can check the folder in Files section of the menu to left-side of screen in colab.

Each test image is used for predictions and is stored as 2 files.

one file is the original ground truth with <filename_gt.jpg>
second file is the prediction of the model on the image. with <filename.jpg>

### Visualize a test image to check predictions


#### Write code to show images: Using cv2.imshow() or matplotlib show any 3 test images and their predictions.

In [0]:
os.chdir("/content/drive/My Drive/ComputerVision_with_CNNs/Face_Detection/output_test")

In [57]:
os.listdir()

['47_Matador_Bullfighter_Matador_Bullfighter_47_617.jpg',
 '47_Matador_Bullfighter_Matador_Bullfighter_47_617_gt.jpg',
 '33_Running_Running_33_475.jpg',
 '33_Running_Running_33_475_gt.jpg',
 '45_Balloonist_Balloonist_45_225.jpg',
 '45_Balloonist_Balloonist_45_225_gt.jpg',
 '21_Festival_Festival_21_395.jpg',
 '21_Festival_Festival_21_395_gt.jpg',
 '51_Dresses_wearingdress_51_113.jpg',
 '51_Dresses_wearingdress_51_113_gt.jpg',
 '20_Family_Group_Family_Group_20_696.jpg',
 '20_Family_Group_Family_Group_20_696_gt.jpg',
 '40_Gymnastics_Gymnastics_40_762.jpg',
 '40_Gymnastics_Gymnastics_40_762_gt.jpg',
 '13_Interview_Interview_Sequences_13_779.jpg',
 '13_Interview_Interview_Sequences_13_779_gt.jpg',
 '4_Dancing_Dancing_4_514.jpg',
 '4_Dancing_Dancing_4_514_gt.jpg',
 '25_Soldier_Patrol_Soldier_Patrol_25_515.jpg',
 '25_Soldier_Patrol_Soldier_Patrol_25_515_gt.jpg',
 '17_Ceremony_Ceremony_17_1048.jpg',
 '17_Ceremony_Ceremony_17_1048_gt.jpg',
 '12_Group_Team_Organized_Group_12_Group_Team_Organized

In [0]:
x_test=[]

import cv2
for i in os.listdir():
  try:
    img = cv2.imread(i) # dtype = uint8
    img = image.img_to_array(img) # dtype = float32
    img = img/255
   
    plt.imshow(img)
  except Exception as e:
    print(e)
   # print(i)

***Not sure why the images are not displaying !!!***